In [22]:
import os

import albumentations as A
import pretrainedmodels
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset

from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from tqdm import tqdm
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.simplefilter('ignore', category=DeprecationWarning)

In [23]:
train_df = pd.read_csv('train.csv')
train_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [24]:
train_df.shape, train_df.Id.nunique()

((25361, 2), 5005)

In [25]:
from albumentations import pytorch as AT
H = 160
W = 320

data_transforms = A.Compose([
    A.Resize(H,W),
    A.HorizontalFlip(),
    A.OneOf([
        A.RandomBrightness(),
        A.RandomContrast()
    ]),
    A.ShiftScaleRotate(rotate_limit=10,scale_limit=.15),
    A.JpegCompression(80),
    A.HueSaturationValue(),
    A.Normalize(),
    AT.ToTensor()
])

data_transforms_test = A.Compose([
    A.Resize(H,W),
    A.Normalize(),
    AT.ToTensor()
])



In [26]:
def prepare_labels(y):
    # From here: https://www.kaggle.com/pestipeti/keras-cnn-starter
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

    y = onehot_encoded
    return y, label_encoder

In [27]:
y, lab_encoder = prepare_labels(train_df['Id'])

In [7]:
class WhaleDataset(Dataset):
    def __init__(self, datafolder, datatype='train', df=None, transform=None, y=None):
        self.datafolder = datafolder
        self.datatype = datatype
        self.y = y
        if self.datatype == 'train':
            self.df = df.values
        self.image_files_list = [s for s in os.listdir(datafolder)]
        self.transform = transform


    def __len__(self):
        return len(self.image_files_list)
    
    def __getitem__(self, idx):
        if self.datatype == 'train':
            img_name = os.path.join(self.datafolder, self.df[idx][0])
            label = self.y[idx]
            
        elif self.datatype == 'test':
            img_name = os.path.join(self.datafolder, self.image_files_list[idx])
            label = np.zeros((NUM_CLASSES,))

        img = cv2.imread(img_name)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image = self.transform(image=img)['image']
        if self.datatype == 'train':
            return image, label
        elif self.datatype == 'test':
            # so that the images will be in a correct order
            return image, label, self.image_files_list[idx]

In [8]:
train_dataset = WhaleDataset(
    datafolder='train/',
    datatype='train',
    df = train_df,
    transform=data_transforms,
    y=y
)

test_set = WhaleDataset(
    datafolder='test/',
    datatype='test',
    transform=data_transforms_test
)

In [9]:
batch_size = 10
train_loader = DataLoader(train_dataset, batch_size=batch_size, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, pin_memory=True)


In [10]:
model = pretrainedmodels.resnet101()

In [11]:
model.avg_pool = nn.AvgPool2d((5,10))

In [12]:
NUM_CLASSES = train_df.Id.nunique()
model.last_linear = nn.Linear(model.last_linear.in_features, NUM_CLASSES)

In [13]:
if torch.cuda.is_available():
    model.cuda()

In [14]:

from torch.optim import lr_scheduler
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(),lr = 1e-3)
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [15]:
def cuda(x):
    return x.cuda(non_blocking=True) if torch.cuda.is_available() else x 

In [16]:
n_epochs = 5
for epoch in range(1, n_epochs+1):
    train_loss = []
    
    for batch_i, (data, target) in tqdm(enumerate(train_loader), total = len(train_loader)):
        data, target = cuda(data), cuda(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.float())
        train_loss.append(loss.item())

        loss.backward()
        optimizer.step()
    
    scheduler.step()
    
    print(f'Epoch {epoch}, train loss: {np.mean(train_loss):.4f}')

  0%|                                                                                         | 0/2537 [00:00<?, ?it/s]

Epoch 1, train loss: 0.0019


  0%|                                                                                         | 0/2537 [00:00<?, ?it/s]

Epoch 2, train loss: 0.0013


  0%|                                                                                         | 0/2537 [00:00<?, ?it/s]

Epoch 3, train loss: 0.0012


  0%|                                                                                         | 0/2537 [00:00<?, ?it/s]

Epoch 4, train loss: 0.0009


100%|██████████████████████████████████████████████████████████████████████████████| 2537/2537 [33:48<00:00,  1.25it/s]


Epoch 5, train loss: 0.0009
